In [1]:
import os
import sys
dir_parent = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(dir_parent) # add parent directory to path
import json
import pickle
import winsound
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
from tqdm import tqdm
from src.db import DataBase
import matplotlib.pyplot as plt
from src.utils_stocks import get_df_c
from src.utils_general import get_df_sym
from src.utils_general import plot_divergences
from src.utils_date import get_ls_date_str_from_db
dir_db = os.path.join(dir_parent, 'data', 'db')
dir_train = os.path.join(dir_parent, 'data', 'train')
db = DataBase([], dir_db=dir_db)

def beeps(n=3):
    '''Produce n beeps. Default 3'''
    for _ in range(n):
        winsound.Beep(200,500)

live_data = 0
target_profit = 0.011
target_loss = -0.031
ls_date_str = get_ls_date_str_from_db('2020-12-14', '2022-01-01', db) #'2020-06-23'
print(ls_date_str[0], ls_date_str[-1])

2020-12-14 2020-12-18


In [4]:
count, count_e = 0, 0
ls_df_t = []
dt_errors = {}
for date_str in ls_date_str:
    print(date_str)
    df_sym = get_df_sym(db, date_str)
    for i, tup in tqdm(df_sym.iterrows(), total=df_sym.shape[0]):
        count+=1
        try:
            df_c = get_df_c(tup['sym'], date_str, live_data, db, target_profit, target_loss)
            ls_df_t.append(df_c[df_c['divergence']!=''])
        except Exception as e:
            dt_errors[tup['sym']] = f'Error: {type(e).__name__} {e}'
            count_e+=1
print('Errors:', count_e, count, round(count_e/count, 3))
print(dt_errors)

2020-12-14


 12%|█████████▏                                                                     | 269/2305 [00:09<01:15, 27.04it/s]


KeyboardInterrupt: 

In [3]:
# df_train - Create dataframe
if ls_df_t:
    df_t = pd.concat(ls_df_t)

# df_train - cleanup columns and NAs
ls_col = [
    'is_profit',
    'rsi14',
    'sma9_var',
    'sma180_var',
    'vwap_var',
    'spread14_e',
    'volume14_34_var',
    'prev_close_var',
    'prev_floor_var',
    'prev_ceil_var',
    'prev1_candle_score',
    'prev2_candle_score',
    'prev3_candle_score',
    'mins_from_start',
    'valley_interval_mins',
    'valley_close_score',
    'valley_rsi_score',
    'day_open_var',
    'open_from_prev_close_var',
    'ceil_var',
    'floor_var',
    ###
    'sym',
    'datetime',
    'profit',
    'prev_close',
    'divergence',
]
df_t = df_t[ls_col]
df_t = df_t.dropna()
#cat_cols = df_t.select_dtypes(include=object).columns
#df_t[cat_cols] = df_t[cat_cols].fillna('none').astype('category')
print(df_t.info())

# df_train - Export
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M')
f = os.path.join(dir_train, f'df_train_{timestamp}.parquet')
df_t.to_parquet(f)
print(f, 'saved!')
beeps()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33747 entries, 22 to 184
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   is_profit                 33747 non-null  bool          
 1   rsi14                     33747 non-null  float64       
 2   sma9_var                  33747 non-null  float64       
 3   sma180_var                33747 non-null  float64       
 4   vwap_var                  33747 non-null  float64       
 5   spread14_e                33747 non-null  float64       
 6   volume14_34_var           33747 non-null  float64       
 7   prev_close_var            33747 non-null  float64       
 8   prev_floor_var            33747 non-null  float64       
 9   prev_ceil_var             33747 non-null  float64       
 10  prev1_candle_score        33747 non-null  float64       
 11  prev2_candle_score        33747 non-null  float64       
 12  prev3_candle_score 

# Debug

In [2]:
date_str = '2020-12-03'
live_data = 1
df_c = get_df_c('BYND', date_str, live_data, db, target_profit, target_loss)
df_c

,datetime,close,sma9,sma180,rsi14,vwap,sma9_var,sma180_var,vwap_var,spread14_e,...,day_open_var,open_from_prev_close_var,ceil,ceil_var,floor,floor_var,sym,actual_buy_price,profit,is_profit
0,2020-12-03 09:30:00,140.699997,138.554767,137.843928,86.772514,140.356664,0.015483,0.020720,0.002446,0.000761,...,0.000000,0.015371,140.699997,0.000000,140.699997,0.000000,BYND,140.960007,None,False
1,2020-12-03 09:31:00,141.644196,138.950789,137.864007,89.744043,140.509232,0.019384,0.027420,0.008078,0.001602,...,0.006711,0.015371,141.644196,0.000000,140.699997,0.006711,BYND,140.750000,None,False
2,2020-12-03 09:32:00,140.960007,139.288567,137.879007,76.357938,140.535142,0.012000,0.022346,0.003023,0.001653,...,0.001848,0.015371,141.644196,-0.004830,140.699997,0.001848,BYND,140.850006,None,False
3,2020-12-03 09:33:00,140.750000,139.576345,137.892674,72.770016,140.560061,0.008409,0.020721,0.001351,0.001461,...,0.000355,0.015371,141.644196,-0.006313,140.699997,0.000355,BYND,140.369995,None,False
4,2020-12-03 09:34:00,140.850006,139.865234,137.907229,73.410743,140.567085,0.007041,0.021339,0.002013,0.001361,...,0.001066,0.015371,141.644196,-0.005607,140.699997,0.001066,BYND,140.492905,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2020-12-03 11:14:00,141.524994,141.528544,140.172950,45.999271,141.756592,-0.000025,0.009646,-0.001634,0.000655,...,0.005864,0.015371,143.250000,-0.012042,140.369995,0.008228,BYND,141.529999,None,False
105,2020-12-03 11:15:00,141.486893,141.481532,140.193323,45.047214,141.756167,0.000038,0.009227,-0.001900,0.000650,...,0.005593,0.015371,143.250000,-0.012308,140.369995,0.007957,BYND,141.800003,None,False
106,2020-12-03 11:16:00,141.529999,141.437088,140.216323,46.398883,141.755768,0.000657,0.009369,-0.001593,0.000610,...,0.005899,0.015371,143.250000,-0.012007,140.369995,0.008264,BYND,141.679398,None,False
107,2020-12-03 11:17:00,141.800003,141.455977,140.240378,54.026739,141.755513,0.002432,0.011121,0.000314,0.000741,...,0.007818,0.015371,143.250000,-0.010122,140.369995,0.010187,BYND,NaN,None,False
